In [8]:
# =========================
# 📌 Cell 1: Setup
# =========================

!pip install ultralytics opencv-python-headless

import cv2
import os
import random
import numpy as np
from ultralytics import YOLO

# Utility functions (simplified versions, replace with your utils if available)
def plot_midpoint(frame, bbox):
    x1, y1, x2, y2 = bbox
    mid = ((x1 + x2) // 2, (y1 + y2) // 2)
    bbox_height = y2 - y1
    cv2.circle(frame, mid, 4, (255, 0, 0), -1)
    return frame, mid, bbox_height

def calculate_distance(p1, p2):
    return np.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

In [12]:
# =========================
# 📌 Cell 2: Upload YOLO model
# =========================
from google.colab import files

print("📂 Upload your YOLOv8 model (.pt)")
uploaded_model = files.upload()
model_path = list(uploaded_model.keys())[0]

# Load YOLO model
model = YOLO(model_path)
class_dict = model.names
print("✅ Model loaded:", model_path)

📂 Upload your YOLOv8 model (.pt)


Saving yolov8m.pt to yolov8m.pt
✅ Model loaded: yolov8m.pt


In [10]:
# =========================
# 📌 Cell 3: Upload video
# =========================
print("📂 Upload your input video (.mp4)")
uploaded_video = files.upload()
video_path = list(uploaded_video.keys())[0]

# Open video
cap = cv2.VideoCapture(video_path)

# Video writer for output
output_path = "output_abandoned_detection.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

📂 Upload your input video (.mp4)


Saving baggage.mp4 to baggage.mp4


In [13]:
# =========================
# 📌 Cell 4: Run Detection + Abandoned Bag Logic
# =========================

colors = [(random.randint(100, 255), random.randint(100, 255), random.randint(100, 255)) for j in range(100)]
detection_threshold = 0.3

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    person_info, bag_info = [], []

    for ind, result in enumerate(results):
        for r in result.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = r
            x1, y1, x2, y2, class_id = map(int, [x1, y1, x2, y2, class_id])
            if score > detection_threshold:
                class_name = class_dict[class_id]
                if class_name == 'person':
                    person_info.append((x1, y1, x2, y2, class_name))
                elif class_name in ['suitcase','backpack','handbag']: # adjust classes if needed
                    bag_info.append((x1, y1, x2, y2, class_name))

    # Draw people
    for i, (x1,y1,x2,y2,class_name) in enumerate(person_info):
        cv2.rectangle(frame, (x1,y1), (x2,y2), colors[i % len(colors)], 3)
        cv2.putText(frame, f'Person-{i}', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)

    # Check bags
    for j, (x1,y1,x2,y2,class_name) in enumerate(bag_info):
        _, mid1, h_bag = plot_midpoint(frame, [x1,y1,x2,y2])
        abandoned = True

        for (a1,b1,a2,b2,_) in person_info:
            _, mid2, h_person = plot_midpoint(frame, [a1,b1,a2,b2])
            if calculate_distance(mid1, mid2) < h_person:  # bag is close to a person
                abandoned = False
                break

        if abandoned:
            cv2.rectangle(frame, (x1,y1), (x2,y2), (0,0,0), 3)
            cv2.putText(frame, "ABANDONED", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
        else:
            cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 3)
            cv2.putText(frame, class_name, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    out.write(frame)

cap.release()
out.release()
print("✅ Processing complete. Saved as:", output_path)

Streaming output truncated to the last 5000 lines.
Speed: 3.2ms preprocess, 13.8ms inference, 1.6ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 3 persons, 1 train, 2 benchs, 2 suitcases, 13.8ms
Speed: 3.2ms preprocess, 13.8ms inference, 1.5ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 3 persons, 1 train, 2 benchs, 2 suitcases, 17.1ms
Speed: 2.9ms preprocess, 17.1ms inference, 2.2ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 3 persons, 1 train, 2 benchs, 2 suitcases, 13.9ms
Speed: 3.4ms preprocess, 13.9ms inference, 1.5ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 3 persons, 1 train, 1 bench, 2 suitcases, 13.6ms
Speed: 3.9ms preprocess, 13.6ms inference, 1.5ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 3 persons, 1 train, 2 benchs, 1 suitcase, 13.4ms
Speed: 3.3ms preprocess, 13.4ms inference, 1.4ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 3 persons, 1 train, 2 benchs, 1 handbag, 1 suit

In [14]:
# =========================
# 📌 Cell 5: Download Output
# =========================
from google.colab import files
files.download(output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>